# 🛠️ LMFast: Agents with Function Calling

**Build agents that can use tools!**

This notebook demonstrates how to give your Small Language Models (SLMs) access to Python functions.

## What You'll Learn
- Defining Python functions as tools
- Creating an `Agent` with tools
- Watching the agent execute tools in a loop


## 1️⃣ Setup

In [ ]:
!pip install -q lmfast[all]

import lmfast
lmfast.setup_colab_env()

import logging
logging.basicConfig(level=logging.INFO)

## 2️⃣ Define Tools

You can use any Python function as a tool. Type hints and docstrings are crucial!

In [ ]:
def calculator(expression: str) -> str:
    """
    Calculate a mathematical expression.
    Args:
        expression: The math expression to evaluate (e.g., '2 + 2', '15 * 24')
    """
    try:
        # Safety: eval is dangerous in prod, ok for demo
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

def weather_lookup(city: str) -> str:
    """
    Get the current weather for a city.
    Args:
        city: The name of the city (e.g., 'New York', 'London')
    """
    # Mock data
    weather_data = {
        "new york": "Sunny, 25°C",
        "london": "Rainy, 15°C",
        "tokyo": "Cloudy, 22°C"
    }
    return weather_data.get(city.lower(), "Weather data not found.")

print("Tools defined!")

## 3️⃣ Create the Agent

We need a model generate function. In a real scenario, you'd use a robust instruction-tuned model.

In [ ]:
from lmfast import Agent
from lmfast.inference import SLMServer

# Load a model capable of following instructions
# SmolLM-135M-Instruct is small but tries its best
model = SLMServer("HuggingFaceTB/SmolLM-135M-Instruct")

def generate_fn(prompt):
    return model.generate(prompt, max_new_tokens=256, temperature=0.1)

# Create Agent with our tools
agent = Agent(
    model_generate_fn=generate_fn,
    tools=[calculator, weather_lookup]
)

print("Agent initialized.")

## 4️⃣ Run the Agent

Let's ask a question that requires a tool.

In [ ]:
query = "What is 1234 * 5678?"
print(f"User: {query}\n")

response = agent.run(query)

print(f"\nFinal Response: {response}")

In [ ]:
query = "What's the weather like in Tokyo?"
print(f"User: {query}\n")

response = agent.run(query)

print(f"\nFinal Response: {response}")